In [1]:
import json
import os
import logging
import pandas as pd
from gensim.models import Word2Vec, Doc2Vec, FastText, KeyedVectors
from gensim.models.phrases import Phraser
from gensim.scripts.glove2word2vec import glove2word2vec
import multiprocessing

number_cpus = multiprocessing.cpu_count()

logging.basicConfig(format="%(asctime)s : %(levelname)s : %(message)s", level=logging.INFO)
logger = logging.getLogger()

data_dir = '/'.join(os.getcwd().split("/")[:-1] + ["data"])
model_dir = '/'.join(os.getcwd().split("/")[:-1] + ["model"])

# _ = glove2word2vec(os.path.join(model_dir, 'english_glove_300'), 
#                   os.path.join(model_dir, 'en_glove_300'))

# Load Data

In [2]:
train_data_en = json.load(open(os.path.join(data_dir, 'train_data_en.json'), 'r'))
train_data_es = json.load(open(os.path.join(data_dir, 'train_data_es.json'), 'r'))
data_en = pd.read_json(os.path.join(data_dir, 'data_en.json'))
data_es = pd.read_json(os.path.join(data_dir, 'data_es.json'))
add_data_1 = json.load(open(os.path.join(model_dir, 'industries_info.json'), 'r'))
add_data_2 = json.load(open(os.path.join(model_dir, 'services_info.json'), 'r'))

In [3]:
data_en

,id,content,title,slug,post_author,post_date,post_date_str,term_slug,term_name,author_name,author_slug_name
0,2869,Are you curious why scalable applications are ...,Scalable Applications: Curious Why Scalability...,why-scalability-matters-for-your-app,8,1616403600000,"Mar 22, 2021",hi-tech,HiTech,Robert Kazmi,robertkazmi
1,2873,A Brief History of Scrum Agile Development: \n...,Is a Scrum Agile Development Process Right for...,is-a-scrum-agile-development-process-right-for...,8,1410739200000,"Sep 15, 2014",hi-tech,HiTech,Robert Kazmi,robertkazmi
2,2876,Guessing is a part of life—and it’s necessary....,How to Make Your App Irresistible Through User...,how-to-make-your-app-irresistible-through-user...,8,1410134400000,"Sep 08, 2014",hi-tech,HiTech,Robert Kazmi,robertkazmi
3,2877,Product management is a tough job. You must tr...,5 Product Manager MUSTS for Creating a Success...,5-product-manager-musts-for-creating-a-success...,8,1409011200000,"Aug 26, 2014",hi-tech,HiTech,Robert Kazmi,robertkazmi
4,2878,From the moment you decide to move forward wit...,7 Key Questions to Ask Your Prospective App De...,7-key-questions-to-ask-prospective-app-develop...,8,1407801600000,"Aug 12, 2014",all-industries,All Industries,Robert Kazmi,robertkazmi
...,...,...,...,...,...,...,...,...,...,...,...
974,7568,"\nWhen it comes to software development, every...",Software Development Best Practices&nbsp;,software-development-best-practices,15,1674234512000,"Jan 20, 2023",app-development,App Development,Jose Gomez,jose-gomez
975,7570,\nPush notifications are a staple of the mobil...,Push Notification as a Service: How Can It Hel...,push-notification-as-a-service,15,1674234887000,"Jan 20, 2023",app-development,App Development,Jose Gomez,jose-gomez
976,7595,\nMany businesses utilize white label app solu...,White Label App Development: Everything You Ne...,white-label-app,8,1674580468000,"Jan 24, 2023",app-development,App Development,Robert Kazmi,robertkazmi
977,7597,\nFinding app development talent can seem daun...,How to Find An App Developer for Your App Deve...,how-to-find-an-app-developer,8,1674580916000,"Jan 24, 2023",app-development,App Development,Robert Kazmi,robertkazmi


In [4]:
data_en["post_date"] = pd.to_datetime(data_en["post_date_str"], infer_datetime_format=True)
data_es["post_date"] = pd.to_datetime(data_es["post_date_str"], infer_datetime_format=True)

# Train w2v model

In [5]:
parameters = {"min_count":0, "size":300, "sg":1, "window":15, "iter":40,
            "sample": 6e-5, "hs": 0, "negative": 15, "ns_exponent": -0.5,
            "workers": number_cpus}#, "max_n":5, "min_n":2, "word_ngrams": 1, "compatible_hash": True}

In [6]:
def train(data, parameters, pretrained_embedding):
    """Train w2v model"""
    model = Word2Vec(**parameters)
    model.build_vocab(data)
#     model.intersect_word2vec_format(os.path.join(model_dir, pretrained_embedding) , binary=False)
    model.train(data, total_examples=model.corpus_count, epochs=model.epochs)
    return model

In [7]:
ft_model_en = train(train_data_en+list(add_data_1.values())+list(add_data_2.values()), 
                    parameters, 'en_glove_300')

2023-01-27 19:30:35,535 : INFO : collecting all words and their counts
2023-01-27 19:30:35,536 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2023-01-27 19:30:35,631 : INFO : collected 16816 word types from a corpus of 486778 raw words and 1029 sentences
2023-01-27 19:30:35,632 : INFO : Loading a fresh vocabulary
2023-01-27 19:30:35,762 : INFO : effective_min_count=0 retains 16816 unique words (100% of original 16816, drops 0)
2023-01-27 19:30:35,763 : INFO : effective_min_count=0 leaves 486778 word corpus (100% of original 486778, drops 0)
2023-01-27 19:30:35,797 : INFO : deleting the raw counts dictionary of 16816 items
2023-01-27 19:30:35,798 : INFO : sample=6e-05 downsamples 1115 most-common words
2023-01-27 19:30:35,802 : INFO : downsampling leaves estimated 276096 word corpus (56.7% of prior 486778)
2023-01-27 19:30:35,846 : INFO : estimated required memory for 16816 words and 300 dimensions: 48766400 bytes
2023-01-27 19:30:35,847 : INFO : resetting la

In [8]:
ft_model_en.wv.similar_by_word("python")

2023-01-27 19:44:55,420 : INFO : precomputing L2-norms of word weight vectors


[('python_python', 0.6990072727203369),
 ('use_python', 0.6868396401405334),
 ('python_code', 0.6553681492805481),
 ('flask', 0.6398506760597229),
 ('sweigart', 0.6263249516487122),
 ('java_python', 0.6246236562728882),
 ('interpreter', 0.6206738948822021),
 ('python_script', 0.6144691705703735),
 ('broader', 0.6110115051269531),
 ('scraping', 0.5993537306785583)]

In [9]:
ft_model_en.wv.similar_by_word("java")

[('bytecode', 0.6942938566207886),
 ('java_virtual', 0.6912535429000854),
 ('java_java', 0.6833294630050659),
 ('appropriateness', 0.6624552011489868),
 ('java_ee', 0.6611155271530151),
 ('use_java', 0.6554378271102905),
 ('fx', 0.6551241874694824),
 ('language_java', 0.6440901756286621),
 ('wora', 0.6414273977279663),
 ('verbose', 0.6373214721679688)]

In [10]:
ft_model_en.wv.similar_by_vector("devop")

[('operation_team', 0.5983641147613525),
 ('devop_philosophy', 0.5664695501327515),
 ('digitalocean', 0.5585602521896362),
 ('devop_team', 0.5561974048614502),
 ('linode', 0.5558598637580872),
 ('heroku', 0.5512608289718628),
 ('deliverymicroservicesinfrastructure', 0.5257720351219177),
 ('agile_devop', 0.5187732577323914),
 ('integrally', 0.516906201839447),
 ('registeringcommunication', 0.515246570110321)]

In [11]:
parameters = {"min_count":0, "size":300, "sg":1, "window":15, "iter":40,
            "sample": 6e-5, "hs": 0, "negative": 15, "ns_exponent": -0.5,
            "workers": number_cpus} #, "max_n":5, "min_n":2, "word_ngrams": 1, "compatible_hash": True}

In [12]:
ft_model_es = train(train_data_es, parameters, 'en_glove_300')

2023-01-27 19:45:04,234 : INFO : collecting all words and their counts
2023-01-27 19:45:04,236 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2023-01-27 19:45:04,238 : INFO : collected 1216 word types from a corpus of 3740 raw words and 10 sentences
2023-01-27 19:45:04,239 : INFO : Loading a fresh vocabulary
2023-01-27 19:45:04,243 : INFO : effective_min_count=0 retains 1216 unique words (100% of original 1216, drops 0)
2023-01-27 19:45:04,244 : INFO : effective_min_count=0 leaves 3740 word corpus (100% of original 3740, drops 0)
2023-01-27 19:45:04,251 : INFO : deleting the raw counts dictionary of 1216 items
2023-01-27 19:45:04,252 : INFO : sample=6e-05 downsamples 1216 most-common words
2023-01-27 19:45:04,253 : INFO : downsampling leaves estimated 1145 word corpus (30.6% of prior 3740)
2023-01-27 19:45:04,257 : INFO : estimated required memory for 1216 words and 300 dimensions: 3526400 bytes
2023-01-27 19:45:04,257 : INFO : resetting layer weights
2023-0

In [13]:
ft_model_es.wv.similar_by_word("koombea")

2023-01-27 19:45:07,499 : INFO : precomputing L2-norms of word weight vectors


[('volumen', 0.9993607997894287),
 ('costoso', 0.9993466734886169),
 ('llamado', 0.9993442296981812),
 ('emprendimiento', 0.999321699142456),
 ('dinero', 0.999312698841095),
 ('plazo', 0.9993088841438293),
 ('abrir', 0.9992982745170593),
 ('mediano', 0.9992853999137878),
 ('personalizar', 0.9992820024490356),
 ('pequeño', 0.9992815852165222)]

In [14]:
ft_model_es.wv.similar_by_word("magento")

[('plantilla', 0.9994137287139893),
 ('ambos', 0.9993163347244263),
 ('usd', 0.9992873668670654),
 ('opción', 0.9990533590316772),
 ('pago', 0.9990230798721313),
 ('plataforma', 0.9989562034606934),
 ('agregar', 0.9987456798553467),
 ('gratuito', 0.9987137317657471),
 ('dominio', 0.998600959777832),
 ('costo', 0.998512864112854)]

In [15]:
ft_model_es.wv.similar_by_word("shopify")

[('pago', 0.9968417882919312),
 ('plataforma', 0.996731162071228),
 ('configurar', 0.9959330558776855),
 ('plan', 0.9958333969116211),
 ('método', 0.9958025813102722),
 ('magento', 0.9957647323608398),
 ('personalización', 0.9956568479537964),
 ('plantilla', 0.9956234693527222),
 ('opción', 0.9955702424049377),
 ('usd', 0.9955036640167236)]

# train fse for each language

In [16]:
from fse.models import SIF, Average
from fse.inputs import IndexedList

In [17]:
fse_data_es = IndexedList(train_data_es)
fse_model_es = Average(ft_model_es, components=0, workers=number_cpus, lang_freq='es')
fse_model_es.train(fse_data_es)

/home/ec2-user/anaconda3/envs/python38/lib/python3.8/site-packages/fse/models/base_s2v.py:114: UserWarning: C extension not loaded, training/inferring will be slow. Install a C compiler and reinstall fse.
  warnings.warn(
2023-01-27 19:45:14,962 : INFO : no frequency mode: using wordfreq for estimation of frequency for language: es
2023-01-27 19:45:15,106 : INFO : scanning all indexed sentences and their word counts
2023-01-27 19:45:15,107 : INFO : finished scanning 10 sentences with an average length of 374 and 3740 total words
2023-01-27 19:45:15,108 : INFO : estimated memory for 10 sentences with 300 dimensions and 1216 vocabulary: 1 MB (0 GB)
2023-01-27 19:45:15,109 : INFO : initializing sentence vectors for 10 sentences
2023-01-27 19:45:15,110 : INFO : begin training
2023-01-27 19:45:15,117 : INFO : worker thread finished; awaiting finish of 1 more threads
2023-01-27 19:45:15,122 : INFO : worker thread finished; awaiting finish of 0 more threads
2023-01-27 19:45:15,123 : INFO : tr

(10, 3740)

In [18]:
fse_data_en = IndexedList(train_data_en)
fse_model_en = Average(ft_model_en, components=0, workers=number_cpus, lang_freq='en')
fse_model_en.train(fse_data_en)

2023-01-27 19:45:16,418 : INFO : no frequency mode: using wordfreq for estimation of frequency for language: en
2023-01-27 19:45:16,557 : INFO : scanning all indexed sentences and their word counts
2023-01-27 19:45:16,559 : INFO : finished scanning 979 sentences with an average length of 496 and 486410 total words
2023-01-27 19:45:16,567 : INFO : estimated memory for 979 sentences with 300 dimensions and 16816 vocabulary: 20 MB (0 GB)
2023-01-27 19:45:16,568 : INFO : initializing sentence vectors for 979 sentences
2023-01-27 19:45:16,573 : INFO : begin training
2023-01-27 19:45:17,101 : INFO : worker thread finished; awaiting finish of 1 more threads
2023-01-27 19:45:17,104 : INFO : worker thread finished; awaiting finish of 0 more threads
2023-01-27 19:45:17,105 : INFO : training on 979 effective sentences with 486410 effective words took 0s with 1839 sentences/s


(979, 486410)

# Use faiss to do the inner product similarity

    1. First normalize matriz
    2. Normalize query
    3. Train faiss index inner product

In [19]:
import faiss
import numpy as np
from faiss import normalize_L2

2023-01-27 19:45:19,967 : INFO : Loading faiss with AVX2 support.
2023-01-27 19:45:20,133 : INFO : Successfully loaded faiss with AVX2 support.


In [20]:
slug_index_es = {slug:index for index, slug in enumerate(data_es["slug"].tolist())}

In [21]:
# Normalize
vectors_es = fse_model_es.sv.vectors
normalize_L2(vectors_es)

# Train idexing innerproduct
index_es = faiss.IndexFlatIP(vectors_es.shape[1])
index_es.add(vectors_es) # add vectors to the index

In [22]:
slug = 'magento-o-shopify-cual-escoger-para-mi-tienda-online'
idx_es = slug_index_es[slug]
top_k = 5
_, I = index_es.search(vectors_es[idx_es].reshape(1, -1), top_k + 1)
response_es = [data_es["slug"][_idx_es] for _idx_es in I.squeeze()[1:]]
print(json.dumps({"relates": response_es}, indent=4))

{
    "relates": [
        "como-usar-shopify-para-navegar-la-crisis",
        "tu-tienda-online-necesita-un-aliado-shopify-plus",
        "un-aliado-shopify-colombia",
        "colombia-mucho-por-aprender-aun-en-temas-de-ecommerce",
        "koombea-emprendimiento-desde-colombia-para-silicon-valley"
    ]
}


In [23]:
slug_index_en = {slug:index for index, slug in enumerate(data_en["slug"].tolist())}

In [24]:
# Normalize
vectors_en = fse_model_en.sv.vectors
normalize_L2(vectors_en)

# Train idexing innerproduct
index_en = faiss.IndexFlatIP(vectors_en.shape[1])
index_en.add(vectors_en) # add vectors to the index

In [25]:
slug = 'what-is-a-full-stack-net-developer'
idx_en = slug_index_en[slug]
top_k = 5
target = vectors_en[idx_en].reshape(1, -1)
_, I = index_en.search(target, top_k + 1)
response_en = [data_en["slug"][_idx_en] for _idx_en in I.squeeze()[1:]]
print(json.dumps({"relates": response_en}, indent=4))

{
    "relates": [
        "a-guide-to-web-development-frameworks",
        "the-best-mobile-app-development-languages-for-your-needs",
        "best-back-end-languages-how-to-choose-the-perfect-one",
        "asp-net-applications",
        "open-source-programming-languages-explained"
    ]
}


In [26]:
data_en.columns

Index(['id', 'content', 'title', 'slug', 'post_author', 'post_date',
       'post_date_str', 'term_slug', 'term_name', 'author_name',
       'author_slug_name'],
      dtype='object')

# Load Phraser

In [27]:
phrases_model_en = Phraser.load(os.path.join(model_dir, "phrases_model_en.pickle"))

2023-01-27 19:45:32,730 : INFO : loading Phraser object from /home/ec2-user/SageMaker/sage-maker/model/phrases_model_en.pickle
2023-01-27 19:45:32,743 : INFO : loaded /home/ec2-user/SageMaker/sage-maker/model/phrases_model_en.pickle


# Services

In [28]:
slug = list(add_data_2.values())[0]
slug = phrases_model_en[slug]
print(slug)
top_k = len(data_en)
target = fse_model_en.infer([(['mvp', 'mvp', 'path', 'launch', 'mvp', 'validate', 'market', 'fit'], 0)])
normalize_L2(target)

V, I = index_en.search(target, 20)
V, I = V.squeeze(), I.squeeze()
# print(V[:5])
response = [(data_en.loc[i]["slug"], int(i), 
             data_en.loc[i]["post_date"].year, data_en.loc[i]["post_date_str"], 
             float(v), data_en.loc[i]["author_slug_name"]) for i, v in zip(I, V)]
temp = response

print(json.dumps({"relates":[respon[0] for respon in response][:3]}, indent=2))
response.sort(key=lambda x: (-x[2], -x[-2]))
response = [respond for respond in response if respond[-1] != "guest-author"]
print(json.dumps({"relates":[(respon[0], f"{respon[-2]:.2f}", respon[-3], respon[-1]) for respon in response][:3]}, indent=2))

2023-01-27 19:45:34,276 : INFO : scanning all indexed sentences and their word counts
2023-01-27 19:45:34,278 : INFO : finished scanning 1 sentences with an average length of 8 and 8 total words


['mvp', 'software_development', 'launch', 'mvp', 'validate', 'product', 'market_fit', 'prioritizing', 'time', 'market', 'essential_feature']
{
  "relates": [
    "example-of-minimum-viable-product",
    "mvp-development-for-startups",
    "how-to-build-a-mvp-as-a-non-tech-founder"
  ]
}
{
  "relates": [
    [
      "example-of-minimum-viable-product",
      "0.79",
      "Dec 21, 2022",
      "robertkazmi"
    ],
    [
      "mvp-development-for-startups",
      "0.77",
      "Sep 08, 2022",
      "robertkazmi"
    ],
    [
      "minimal-marketable-product",
      "0.71",
      "Aug 11, 2022",
      "robertkazmi"
    ]
  ]
}


# Industry

In [29]:
add_data_1.keys()

dict_keys(['fintech-app-development-services', 'custom-healthcare-software-development-services', 'hitech', 'retail-app-development-services', 'iot-app-development-services', 'educational-app-development-services'])

In [30]:
slug = list(add_data_1.values())[4]
slug = phrases_model_en[slug]
print(slug)
top_k = len(data_en)
target = fse_model_en.infer([(slug, 0)])
normalize_L2(target)

V, I = index_en.search(target, 20)
V, I = V.squeeze(), I.squeeze()
print(V[:5])
response = [(data_en.loc[i]["slug"], int(i), 
             data_en.loc[i]["post_date"].year, data_en.loc[i]["post_date_str"], 
             float(v), "".join(data_en.loc[i]["term_slug"].split("-")), data_en.loc[i]["author_slug_name"]) for i, v in zip(I, V)]
temp = response
print(json.dumps({"relates":[respon[0] for respon in response][:3]}, indent=2))
response.sort(key=lambda x: (-x[2], -x[-3]))

if slug[0] in data_en["slug"].values:
    response = [respon for respon in response if respon[-2] == slug[0]]

response = [respon for respon in response if respon[-1] != "guest-author"]

    
print(json.dumps({"relates":[(respon[0], f"{respon[-3]:.2f}", respon[-4], respon[-2], respon[-1]) for respon in response][:3]}, indent=2))

2023-01-27 19:45:37,868 : INFO : scanning all indexed sentences and their word counts
2023-01-27 19:45:37,869 : INFO : finished scanning 1 sentences with an average length of 3 and 3 total words


['iot', 'development', 'service']
[0.83503044 0.8306978  0.8179548  0.8049939  0.7955551 ]
{
  "relates": [
    "iot-security-issues",
    "iot-changing-everything",
    "the-future-belongs-to-iot-your-strategy-should-reflect-it"
  ]
}
{
  "relates": [
    [
      "what-is-cloud-application-development",
      "0.76",
      "Nov 03, 2022",
      "hitech",
      "jtarud"
    ],
    [
      "iot-security-issues",
      "0.84",
      "Apr 26, 2021",
      "iot",
      "jose-gomez"
    ],
    [
      "mobile-iot",
      "0.79",
      "Aug 18, 2021",
      "iot",
      "jose-gomez"
    ]
  ]
}


In [31]:
add_data_2["machine-learning"]

['machine',
 'learn',
 'artificial',
 'intelligence',
 'data',
 'management',
 'deep',
 'learning',
 'mlop']

# Save the models

In [32]:
fse_model_en.save(os.path.join(model_dir, 'fse_model_en.pickle'))

2023-01-27 19:45:42,035 : INFO : saving Average object under /home/ec2-user/SageMaker/sage-maker/model/fse_model_en.pickle, separately None
2023-01-27 19:45:42,416 : INFO : saved /home/ec2-user/SageMaker/sage-maker/model/fse_model_en.pickle


In [33]:
fse_model_es.save(os.path.join(model_dir, 'fse_model_es.pickle'))

2023-01-27 19:45:42,916 : INFO : saving Average object under /home/ec2-user/SageMaker/sage-maker/model/fse_model_es.pickle, separately None
2023-01-27 19:45:42,931 : INFO : saved /home/ec2-user/SageMaker/sage-maker/model/fse_model_es.pickle


# Test local endpoint

In [34]:
add_data_1.keys(), add_data_2.keys()

(dict_keys(['fintech-app-development-services', 'custom-healthcare-software-development-services', 'hitech', 'retail-app-development-services', 'iot-app-development-services', 'educational-app-development-services']),
 dict_keys(['mvp-software-development', 'product-ideation', 'quality-assurance', 'product-design', 'ecommerce-development-solutions', 'project-management', 'devops', 'cross-platform-app-development', 'web-development', 'android-app-development', 'ios-app-development', 'machine-learning', 'staff-augmentation', 'pwa', 'custom-mobile-app-development', 'product-discovery', 'web-app-design', 'mobile-app-design', 'web-app-development', 'website-redesign', 'application-modernization', 'cloud-application-development', 'it-consulting', 'nft-marketplace-development', 'enterprise-application-development', 'devops-consulting', 'blockchain-consulting', 'saas-development', 'big-data', 'cloud-consulting', 'software-development-solutions', 'shopify-development', 'augmented-reality', 'eco

In [35]:
import requests
import json

!curl --location --request GET http://0.0.0.0:8080/ping

results = {}

for key_val in list(add_data_1.keys()) + list(add_data_2.keys()):
    url = "http://0.0.0.0:8080/invocations"
    body = {
                "slug": key_val, 
                "lang": "en",
                "topn": 3,
                "slug_blocked": None
            }
    body = json.dumps(body)
    headers = {
      'Content-Type': 'application/json'
    }
    response = requests.request("POST", url, headers=headers, data = body)
    text_response = json.loads(response.text)
    results[key_val] = text_response
# print(json.dumps(text_response, indent=4))

print(json.dumps(results, indent=4))

{"status":"ok"}{
    "fintech-app-development-services": {
        "relates": [
            "how-to-make-a-fintech-app",
            "how-to-start-a-fintech-company",
            "fintech-app-design"
        ]
    },
    "custom-healthcare-software-development-services": {
        "relates": [
            "customized-business-apps",
            "costs-of-software-development",
            "how-to-create-a-medical-app"
        ]
    },
    "hitech": {
        "relates": [
            "how-hitech-organizations-use-ai-machine-learning",
            "signs-you-need-hitech-app-development",
            "finding-the-perfect-hitech-app-development-partner"
        ]
    },
    "retail-app-development-services": {
        "relates": [
            "retail-application",
            "what-is-the-difference-between-ecommerce-and-ebusiness",
            "app-development-cost"
        ]
    },
    "iot-app-development-services": {
        "relates": [
            "what-is-cloud-application-developme

In [36]:
!curl --location --request GET http://0.0.0.0:8080/ping
print()
import requests
import json


results = {}
slug_blocked = ["the-difference-between-a-qa-analyst-and-qa-engineer"]

# key_val = "trending-blogs-api"
# key_val = "popular-blogs-api"
# key_val = "what-is-machine-learning"
# key_val = "the-difference-between-a-qa-analyst-and-qa-engineer"
key_val = "quality-assurance"
# key_val = 'qa-quality-assurance'
url = "http://0.0.0.0:8080/invocations"
body = {
            "slug": key_val, 
            "lang": "en",
            "topn": 3,
            "slug_blocked": slug_blocked 
        }
body = json.dumps(body)
headers = {
  'Content-Type': 'application/json'
}
response = requests.request("POST", url, headers=headers, data = body)

text_response = json.loads(response.text)
results[key_val] = text_response
print(json.dumps(text_response, indent=4))

{"status":"ok"}
{
    "relates": [
        "quality-assurance-vs-quality-control",
        "the-importance-of-quality-assurance-in-app-development",
        "qa-auditing"
    ]
}
